#### Import block

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import Constant

%matplotlib inline

In [2]:
# Check for available GPU.
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

#### Function definitions

In [3]:
# Loader functions
# Inspiration: https://towardsdatascience.com/writing-custom-keras-generators-fe815d992c5a

def get_input(path):
    # Load array.
    t_input = np.load(path)
    # Move channel axis to the end.
    t_input = np.moveaxis(t_input, 0, -1)
    # Pad to even number of pixels
    t_input = np.pad(t_input, [(0,0),(0,1),(0,1),(0,0)])
    return t_input

def get_output(path):
    # Load array.
    t_output = np.load(path)
    # Pad to even number of pixels
    #t_output = np.pad(t_output, [(0,0),(0,1),(0,1)])
    # Resize to include a channel dimension.
    #t_output = tf.expand_dims(t_output, axis = -1)
    t_output = t_output.sum()
    return t_output

def data_generator(samples, batch_size = 64):
    
    while True:
        # Select files (paths/indices) for the batch
        batch_samples  = np.random.choice(a = samples.index, 
                                      size = batch_size)
        batch_input  = []
        batch_output = [] 

        # Read in each input, perform preprocessing and get labels
        for sample in batch_samples:
          input = get_input(samples.loc[sample].features)
          output = get_output(samples.loc[sample].labels)

          batch_input += [input]
          batch_output += [output]
        # Return a tuple of (input, output) to feed the network
        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)
        
        yield(batch_x, batch_y)


In [4]:
m = pd.read_csv('Sample_Dataset_2/train/meta.csv')
m

,Unnamed: 0,Lon,Lat,Date,features,labels
0,356962,-53.345535,-6.535028,2017-07-30,Sample_Dataset_2/train/features/356962.npy,Sample_Dataset_2/train/labels/356962.npy
1,546517,-47.329685,-8.260676,2017-08-20,Sample_Dataset_2/train/features/546517.npy,Sample_Dataset_2/train/labels/546517.npy
2,799359,-50.967861,-8.255809,2017-09-04,Sample_Dataset_2/train/features/799359.npy,Sample_Dataset_2/train/labels/799359.npy
3,714590,-66.144379,-12.624272,2017-08-31,Sample_Dataset_2/train/features/714590.npy,Sample_Dataset_2/train/labels/714590.npy
4,240012,-48.350338,-11.838207,2017-07-14,Sample_Dataset_2/train/features/240012.npy,Sample_Dataset_2/train/labels/240012.npy
...,...,...,...,...,...,...
6491,173827,-54.326607,-11.983384,2017-06-20,Sample_Dataset_2/train/features/173827.npy,Sample_Dataset_2/train/labels/173827.npy
6492,1377764,-47.425915,-7.486426,2017-09-26,Sample_Dataset_2/train/features/1377764.npy,Sample_Dataset_2/train/labels/1377764.npy
6493,1444041,-65.172768,-10.714162,2017-10-02,Sample_Dataset_2/train/features/1444041.npy,Sample_Dataset_2/train/labels/1444041.npy
6494,644793,-53.701656,-15.977278,2017-08-27,Sample_Dataset_2/train/features/644793.npy,Sample_Dataset_2/train/labels/644793.npy


#### Load data

In [5]:
# Get data from one chip
sample_input = np.load('Sample_Dataset_2/train/features/3243.npy')

# Move channel axis to the end.
sample_input = np.moveaxis(sample_input, 0, -1)
# Pad to even number of pixels
a = np.pad(sample_input, [(0,0),(0,1),(0,1),(0,0)])
# Resize to include a batch dimension.
a = tf.expand_dims(a, axis = 0)
# Display shape for verification.
a.shape

TensorShape([1, 4, 32, 32, 4])

#### Experiment with basic model structure

In [6]:
b = tf.keras.layers.Conv3D(64, (3,3,3), padding = 'same', activation='relu', bias_initializer=Constant(0.01), 
                           input_shape=(a))(a)
b.shape

TensorShape([1, 4, 32, 32, 64])

In [7]:
c = tf.keras.layers.Conv3D(64, (3,3,3), padding = 'same', activation='relu', bias_initializer=Constant(0.01) 
                           )(b)
c.shape

TensorShape([1, 4, 32, 32, 64])

In [8]:
d = layers.MaxPooling3D((2,2,2))(c)
d.shape

TensorShape([1, 2, 16, 16, 64])

In [9]:
e = tf.keras.layers.Conv3D(128, (3,3,3), padding = 'same', activation='relu')(d)
e.shape

TensorShape([1, 2, 16, 16, 128])

In [10]:
f = tf.keras.layers.Conv3D(128, (3,3,3), padding = 'same', activation='relu')(e)
f.shape

TensorShape([1, 2, 16, 16, 128])

In [11]:
g = layers.MaxPooling3D((1,2,2))(f)
g.shape

TensorShape([1, 2, 8, 8, 128])

In [12]:
h = tf.keras.layers.Conv3D(256, (3,3,3), padding = 'same', activation='relu')(g)
h.shape

TensorShape([1, 2, 8, 8, 256])

In [13]:
i = tf.keras.layers.Conv3D(256, (3,3,3), padding = 'same', activation='relu')(h)
i.shape

TensorShape([1, 2, 8, 8, 256])

In [14]:
j = layers.MaxPooling3D((1,2,2))(i)
j.shape

TensorShape([1, 2, 4, 4, 256])

In [15]:
k = tf.keras.layers.Conv3D(512, (3,3,3), padding = 'same', activation='relu')(j)
k.shape

TensorShape([1, 2, 4, 4, 512])

In [16]:
l = tf.keras.layers.Conv3D(512, (3,3,3), padding = 'same', activation='relu')(k)
l.shape

TensorShape([1, 2, 4, 4, 512])

In [17]:
m = layers.MaxPooling3D((2,4,4))(l)
m.shape

TensorShape([1, 1, 1, 1, 512])

In [18]:
n = layers.Flatten()(m)
n.shape

TensorShape([1, 512])

In [19]:
o = layers.Dense(64, activation='relu')(n)
o.shape

TensorShape([1, 64])

In [20]:
p = layers.Dense(32, activation='relu')(o)
p.shape

TensorShape([1, 32])

In [21]:
q = layers.Dense(16, activation='relu')(p)
q.shape

TensorShape([1, 16])

In [22]:
r = layers.Dense(1, activation='linear')(q)
r.shape

TensorShape([1, 1])

#### Model assumbly

In [23]:
inputs = layers.Input(shape=((4,32,32,4)))
b = tf.keras.layers.Conv3D(32, (3,3,3), padding = 'same', activation='gelu', bias_initializer=Constant(0.01))(inputs)
c = tf.keras.layers.Conv3D(32, (3,3,3), padding = 'same', activation='gelu', bias_initializer=Constant(0.01))(b)
d = layers.MaxPooling3D((2,2,2))(c)
d = layers.BatchNormalization()(d)
d = layers.Dropout(0.3)(d)
e = tf.keras.layers.Conv3D(64, (3,3,3), padding = 'same', activation='gelu')(d)
f = tf.keras.layers.Conv3D(64, (3,3,3), padding = 'same', activation='gelu')(e)
g = layers.MaxPooling3D((1,2,2))(f)
g = layers.BatchNormalization()(g)
g = layers.Dropout(0.3)(g)
h = tf.keras.layers.Conv3D(128, (3,3,3), padding = 'same', activation='gelu')(g)
i = tf.keras.layers.Conv3D(128, (3,3,3), padding = 'same', activation='gelu')(h)
j = layers.MaxPooling3D((1,2,2))(i)
j = layers.BatchNormalization()(j)
j = layers.Dropout(0.3)(j)
k = tf.keras.layers.Conv3D(256, (3,3,3), padding = 'same', activation='gelu')(j)
l = tf.keras.layers.Conv3D(256, (3,3,3), padding = 'same', activation='gelu')(k)
m = layers.MaxPooling3D((2,4,4))(l)
m = layers.BatchNormalization()(m)
m = layers.Dropout(0.3)(m)
n = layers.Flatten()(m)
o = layers.Dense(1024, activation='gelu')(n)
o = layers.BatchNormalization()(o)
o = layers.Dropout(0.3)(o)
p = layers.Dense(1024, activation='gelu')(o)
p = layers.BatchNormalization()(p)
p = layers.Dropout(0.3)(p)
q = layers.Dense(1024, activation='gelu')(p)
q = layers.BatchNormalization()(q)
q = layers.Dropout(0.3)(q)
outputs = layers.Dense(1, activation='linear')(q)

In [24]:
forecast_model = tf.keras.Model(inputs, outputs, name="3D_CNN")

In [25]:
# Verify output shape.
forecast_model.predict(a).shape

1/1 [==============================] - 0s 194ms/step


(1, 1)

In [26]:
# Display model details.
forecast_model.summary()

Model: "3D_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4, 32, 32, 4)]    0         
                                                                 
 conv3d_8 (Conv3D)           (None, 4, 32, 32, 32)     3488      
                                                                 
 conv3d_9 (Conv3D)           (None, 4, 32, 32, 32)     27680     
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 2, 16, 16, 32)    0         
 3D)                                                             
                                                                 
 batch_normalization (BatchN  (None, 2, 16, 16, 32)    128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 2, 16, 16, 32)     0    

In [27]:
# Compile model.
opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
forecast_model.compile(loss = tf.keras.losses.MeanSquaredError(), 
                       optimizer=opt, 
                       metrics = [tf.keras.metrics.MeanSquaredError()])

In [28]:
# Data loaders.
batch_size = 64
meta_t = pd.read_csv('Sample_Dataset_2/train/meta.csv')
meta_v = pd.read_csv('Sample_Dataset_2/val/meta.csv')
t_gen = data_generator(meta_t, batch_size = batch_size)
v_gen = data_generator(meta_v, batch_size = batch_size)

In [29]:
# Train model.
forecast_model.fit(t_gen, 
                   epochs = 50, 
                   verbose = 1, 
                   validation_data = v_gen, 
                   steps_per_epoch = len(meta_t) // batch_size,
                   validation_steps = len(meta_v) // batch_size,
                  )

Epoch 1/50
101/101 [==============================] - 45s 433ms/step - loss: 427.5701 - mean_squared_error: 427.5701 - val_loss: 268.1029 - val_mean_squared_error: 268.1029
Epoch 2/50
101/101 [==============================] - 20s 203ms/step - loss: 409.1885 - mean_squared_error: 409.1885 - val_loss: 300.0758 - val_mean_squared_error: 300.0758
Epoch 3/50
101/101 [==============================] - 12s 116ms/step - loss: 404.4841 - mean_squared_error: 404.4841 - val_loss: 357.5589 - val_mean_squared_error: 357.5589
Epoch 4/50
101/101 [==============================] - 9s 90ms/step - loss: 389.3833 - mean_squared_error: 389.3833 - val_loss: 338.6359 - val_mean_squared_error: 338.6359
Epoch 5/50
101/101 [==============================] - 8s 75ms/step - loss: 404.1831 - mean_squared_error: 404.1831 - val_loss: 309.6389 - val_mean_squared_error: 309.6389
Epoch 6/50
101/101 [==============================] - 7s 71ms/step - loss: 390.4630 - mean_squared_error: 390.4630 - val_loss: 512.6423 - v

Epoch 49/50
101/101 [==============================] - 7s 69ms/step - loss: 303.6332 - mean_squared_error: 303.6332 - val_loss: 237.0814 - val_mean_squared_error: 237.0814
Epoch 50/50
101/101 [==============================] - 7s 69ms/step - loss: 325.6676 - mean_squared_error: 325.6676 - val_loss: 320.7943 - val_mean_squared_error: 320.7943


In [30]:
# Save trained model.
forecast_model.save('Models/006')

INFO:tensorflow:Assets written to: Models/005\assets


INFO:tensorflow:Assets written to: Models/005\assets


#### Examine how model is working

In [37]:
# Get data from one chip
sample_input = np.load('Sample_Dataset_2/val/features/2144676.npy')

# Move channel axis to the end.
sample_input = np.moveaxis(sample_input, 0, -1)
# Pad to even number of pixels
a = np.pad(sample_input, [(0,0),(0,1),(0,1),(0,0)])
# Resize to include a batch dimension.
a = tf.expand_dims(a, axis = 0)

# Get data from one chip
sample_output = np.load('Sample_Dataset_2/val/labels/2144676.npy')

# Pad to even number of pixels
b = np.pad(sample_output, [(0,0),(0,1),(0,1)])
# Resize to include a channel dimension.
#b = tf.expand_dims(b, axis = -1)
# Resize to include a batch dimension.
#b = tf.expand_dims(b, axis = 0)
b.sum()

2.0

In [38]:
pred_eval = forecast_model.predict(a)
pred_eval[0][0]

1/1 [==============================] - 0s 15ms/step


-8.827546

In [40]:
f_list = [
    '2144676.npy',
    '2200301.npy',
    '2279930.npy',
    '2323300.npy',
    '2365430.npy',
    '2461436.npy',
    '2862603.npy',
    '3159008.npy',
    '3382322.npy',
    '3439800.npy'
]

for x in range(0, 10):
    a = np.load('Sample_Dataset_2/val/features/' + f_list[x])
    a = np.moveaxis(a, 0, -1)
    a = np.pad(a, [(0,0),(0,1),(0,1),(0,0)])
    a = tf.expand_dims(a, axis = 0)
    b = np.load('Sample_Dataset_2/val/labels/' + f_list[x])
    pred_eval = forecast_model.predict(a)
    print(b.sum())
    print(pred_eval[0][0])
    print()

1/1 [==============================] - 0s 14ms/step
2.0
-8.827546

1/1 [==============================] - 0s 14ms/step
0.0
1.305102

1/1 [==============================] - 0s 14ms/step
0.0
-8.8289585

1/1 [==============================] - 0s 14ms/step
0.0
-8.836891

1/1 [==============================] - 0s 14ms/step
0.0
-6.927685

1/1 [==============================] - 0s 14ms/step
0.0
1.8310685

1/1 [==============================] - 0s 14ms/step
3
-6.2787557

1/1 [==============================] - 0s 14ms/step
32
-1.2721663

1/1 [==============================] - 0s 14ms/step
0.0
-7.969244

1/1 [==============================] - 0s 15ms/step
1.0
7.294002

